## Image Classification

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0, VGG19
from tensorflow.keras.applications.efficientnet import decode_predictions

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Softmax, BatchNormalization, Input
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, LambdaCallback
from keras.layers.experimental import preprocessing

from keras.optimizers import schedules

import util

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_img_dir='../../data/train'
test_img_dir='../../data/test'
train_csv_path = '../../data/train.csv'
test_csv_path = '../../data/sample_submission.csv'

In [5]:
train = pd.read_csv(train_csv_path)
test = pd.read_csv(test_csv_path)

In [6]:
train.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [7]:
# config settings
class CONFIG:
    # image loading
    img_width = 224
    img_height = 224

    # nn training
    epochs = 100
    batch_size = 128
    lr = 1e-5
    
    # callbacks
    checkpoint_filepath = './temp/checkpoint'

In [8]:
# DEBUG only: sample subset of the data
train_size = 10000

In [9]:
X_train = util.load_img_RGB(train_img_dir,train.Image[:train_size], CONFIG.img_width, CONFIG.img_height)
X_train /= 255
y_train = np.array(train.Id[:train_size])

In [10]:
# encode y
y_train_encoded, label_encoder = util.prepare_labels(y_train)

In [11]:
# try stratified split, if doesn't work, use train_test_split
try:
    X_train, y_train, X_val, y_val = util.split(X_train,y_train_encoded,0.2)
except:
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train_encoded,test_size=0.2)

## Transfer Learning

In [12]:
dense_num_nodes = y_train_encoded.shape[1]

In [13]:
# load pretrained layers
efn_pretrained = EfficientNetB0(weights='imagenet',
                                include_top=False,
                                input_shape=(CONFIG.img_width, CONFIG.img_height,3))
for layer in efn_pretrained.layers:
    layer.trainable = False

In [14]:
# add top layers

#new_layers = GlobalAveragePooling2D(name="avg_pool")(efn_pretrained.output)
new_layers = Flatten()(efn_pretrained.output)
new_layers = Dense(1024, kernel_initializer="he_normal", activation="relu")(new_layers)
new_layers = Dense(1024, kernel_initializer="he_normal", activation="relu")(new_layers)
new_layers = BatchNormalization()(new_layers)
new_layers = Dropout(0.5)(new_layers)
new_layers = Dense(dense_num_nodes, kernel_initializer="he_normal", activation="relu")(new_layers)
predictions = Softmax()(new_layers)
efn_whale = Model(efn_pretrained.input, predictions)

#### Callbacks

In [15]:
# scheduler: a few options
scheduler_exp = schedules.ExponentialDecay(0.0001, decay_steps=100000, decay_rate=1e-6)
#scheduler_inverse = schedules.InverseTimeDecay()
#scheduler_poly = schedules.PolynomialDecay()
#scheduler_piecewise = schedules.PiecewiseConstantDecay()

In [16]:
# optimizer: a few options
#adam = optimizers.Adam(CONFIG.lr,clipnorm=0.1)
adam = optimizers.Adam(scheduler_exp,clipnorm=0.1)
#nadam = optimizers.Nadam(CONFIG.lr)

In [17]:
# FIX: logs.get('weights') - dn work right now
class WeightHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.weights = [] 
    def on_epoch_end(self, batch, logs={}):
       self.weights.append(logs.get('weights'))
#       self.weights.append(efn_shopee.layers[242].get_weights())

In [18]:
weight_hist = WeightHistory()

In [19]:
callbacks = [
    EarlyStopping(patience=20,restore_best_weights=True)
    # ModelCheckpoint(filepath=CONFIG.checkpoint_filepath,save_best_only=True),
    # print_weights = LambdaCallback(on_batch_begin=lambda batch, logs: print(efn_shopee.layers[242].get_weights()))
]

------------

In [20]:
# probably should be 'categorical_crossentropy' because y is one hot encoded
efn_whale.compile(loss = "sparse_categorical_crossentropy", 
                  optimizer = adam, 
                  metrics=["accuracy"])

In [ ]:
history = efn_whale.fit(
    X_train, y_train, 
    batch_size=CONFIG.batch_size, epochs=CONFIG.epochs, verbose=1,
    shuffle=True, validation_data=(X_val,y_val),
    callbacks=callbacks)

Epoch 1/100
